# Optical Flow
    1. Optical flow is the pattern of apparent motion of image objects between two consecutive          frames caused by the movement of object or camera
    2. It often serves as a good approximation of the true physical motion projected onto the          image plane
    3. Optical flow provides a concise description of both the regions of the image undergoing          motion and the velocity of motion.
    
    
  Refer table 19.1  Comparision over motion changes detection tools
    
  Refrence 1. Advances in Video-Based Human Activity Analysis: Challenges and Approaches                       Pavan Turaga Rama Chellappa Ashok Veera raghavan


NOTE: It is probably a good idea to restart the kernel if you ever run these cells, as the tracking algo can sometimes get caught in a loop with your camera.


In [5]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
corner_track_params = dict(maxCorners = 10,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

### have Look at optical pyramid for image processing 

   image resolution is imortant as shown in image pyramid in Optical flow 
Refer : https://en.wikipedia.org/wiki/Pyramid_(image_processing)#/media/File:Image_pyramid.svg

In [7]:
# prameters for lukas kanade otical flow prameters

lukas_kanade_prams = dict(winSize=(200,200),maxLevel=2,criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.03))

In [8]:
# Read video data
capture_video = cv2.VideoCapture(0)
ret, previous_frame = capture_video.read()

# Read in gray scale
previous_frame_gray = cv2.cvtColor(previous_frame,cv2.COLOR_BGR2GRAY)


# set points to Track
# Note ** = laws to pass dictionary of corner points 
prev_points = cv2.goodFeaturesToTrack(previous_frame_gray, mask = None, **corner_track_params)

mask = np.zeros_like(previous_frame)


while True:
    
    # read for  current frame
    ret, current_frame = capture_video.read()
    
    # gray scale read for current frame
    current_frame_gray = cv2.cvtColor(current_frame,cv2.COLOR_BGR2GRAY)
    
    # Now calculate optical flow on the gray scale
    # get next point, status and error using pyramid lukas kanade using following function
    
    nextPts , status, err = cv2.calcOpticalFlowPyrLK(previous_frame_gray,current_frame_gray,prev_points,None,**lukas_kanade_prams)
    
    # lets return the status array 
    
    next_new_pts = nextPts[status == 1]
    prev_new_pts = prev_points[status == 1]
    
    for i, (new,prev) in enumerate(zip(next_new_pts,prev_new_pts)):
        
        # set x and y points to draw with np.ravel option
        x_new,y_new = new.ravel()
        x_prev,y_prev = prev.ravel()
        
        mask = cv2.line(mask,(x_new,y_new),(x_prev,y_prev),(0,255,0),3)
        
        frame = cv2.circle(current_frame,(x_new,y_new),8,(255,0,0),1)
        
        
        
    img = cv2.add(frame,mask)
    cv2.imshow('Tracking',img)
    
    
    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break
        
        
# update previous frame for next iteration

    previous_frame_gray = current_frame_gray.copy()
    prev_points = next_new_pts.reshape(-1,1,2)
    
    
    
cv2.destroyAllWindows()
capture_video.release()
    